In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import pearsonr, spearmanr
%matplotlib inline

In [2]:
df = pd.read_csv('alpha24_step15Stud.csv')
df.head()

,czas(min),alpha,temp,Ts
0,0,1,"6,50",2
1,15,0,"6,35","2,18"
2,30,0,"6,20","2,19"
3,45,1,"6,05","1,68"
4,60,1,"5,90","1,5"


In [3]:
df['temp'] = df['temp'].apply(lambda x: x.replace(',','.')).astype('float64')
df['Ts'] = df['Ts'].apply(lambda x: x.replace(',','.')).astype('float64')

In [4]:
df.head()

,czas(min),alpha,temp,Ts
0,0,1,6.50,2.00
1,15,0,6.35,2.18
2,30,0,6.20,2.19
3,45,1,6.05,1.68
4,60,1,5.90,1.50


In [5]:
df.rename(columns={"czas(min)": "time", "alpha": "compressor", "temp": 'ambient_temp', "Ts": 'refrigerator_temp'}, inplace=True)
df.set_index('time', inplace=True)
df.head()

,compressor,ambient_temp,refrigerator_temp
time,,,
0,1,6.50,2.00
15,0,6.35,2.18
30,0,6.20,2.19
45,1,6.05,1.68
60,1,5.90,1.50


In [6]:
size = 1000
delay = 100

a = df.iloc[0:size,:].to_numpy().reshape(delay,size*3//delay)
a

array([[ 1.  ,  6.5 ,  2.  , ...,  0.  ,  5.26,  1.82],
       [ 0.  ,  5.23,  1.86, ...,  0.  ,  5.39,  1.48],
       [ 0.  ,  5.3 ,  1.53, ...,  1.  ,  5.99,  0.9 ],
       ...,
       [ 0.  ,  1.97, -1.98, ...,  0.  ,  1.23, -1.93],
       [ 0.  ,  1.2 , -1.92, ...,  0.  ,  2.01, -2.07],
       [ 1.  ,  2.13, -2.17, ...,  1.  ,  4.01, -2.48]])

In [7]:
import random_forest
from random_forest.tree import Tree

random_forest.tree.config.min_split_size = 30
random_forest.tree.config.number_of_predictors_to_draw = 10

In [8]:
myTree = Tree(a)
# test_data = df.iloc[1000:1010,:].to_numpy().reshape(30)
print(test_data, myTree.predict(test_data))

NameError: name 'test_data' is not defined

In [9]:
print(myTree)


        min_split_size: 30
        number_of_predictors_to_draw: 10
        
        Branch:
            if <= 23_-1.09:
                
        Branch:
            if <= 26_-2.21:
                Leaf: -2.6404761904761904
            else:
                
        Branch:
            if <= 17_-1.49:
                Leaf: -1.8777272727272725
            else:
                Leaf: -1.30375  
          
        
            else:
                
        Branch:
            if <= 23_-0.15:
                Leaf: -0.6383333333333333
            else:
                Leaf: 0.6175999999999999  
          
        
        


In [10]:
from random_forest.forest import Forest
myForest = Forest(a)

myForest.trees[0]


        min_split_size: 30
        number_of_predictors_to_draw: 10
        
        Branch:
            if <= 26_-1.15:
                
        Branch:
            if <= 26_-2.21:
                Leaf: -2.6404761904761904
            else:
                
        Branch:
            if <= 17_-1.49:
                Leaf: -1.8777272727272725
            else:
                Leaf: -1.3162500000000001  
          
        
            else:
                
        Branch:
            if <= 20_-0.18:
                Leaf: -0.5788461538461539
            else:
                Leaf: 0.663913043478261  
          
        
        

In [20]:


delay = 3
frames = [df.shift(-t) for t in range(delay)]
# small_df = df.iloc[:40]
delayed_df = pd.concat(frames, axis=1).dropna()

small_df = delayed_df.iloc[:20]
small_df.shape


(20, 9)

In [24]:
out_of_bag  = small_df
print(out_of_bag.size)
data_size_for_tree = 5
training_data = small_df.sample(data_size_for_tree, replace=True)
out_of_bag = out_of_bag[~out_of_bag.index.isin(training_data.index)]
# out_of_bag[~out_of_bag.isin(training_data)].dropna()
# out_of_bag.isin(training_data.merge(training_data))
# print(out_of_bag.size)
training_data.head()


180


,compressor,ambient_temp,refrigerator_temp,compressor,ambient_temp,refrigerator_temp,compressor,ambient_temp,refrigerator_temp
time,,,,,,,,,
45,1,6.05,1.68,1.0,5.90,1.50,0.0,5.74,1.64
90,0,5.59,1.62,0.0,5.44,1.56,0.0,5.30,1.54
255,0,5.60,1.22,0.0,5.49,1.22,0.0,5.39,1.48
165,0,5.21,1.89,1.0,5.20,1.20,1.0,5.32,1.20
45,1,6.05,1.68,1.0,5.90,1.50,0.0,5.74,1.64


In [25]:
out_of_bag


,compressor,ambient_temp,refrigerator_temp,compressor,ambient_temp,refrigerator_temp,compressor,ambient_temp,refrigerator_temp
time,,,,,,,,,
0,1,6.50,2.00,0.0,6.35,2.18,0.0,6.20,2.19
15,0,6.35,2.18,0.0,6.20,2.19,1.0,6.05,1.68
30,0,6.20,2.19,1.0,6.05,1.68,1.0,5.90,1.50
60,1,5.90,1.50,0.0,5.74,1.64,0.0,5.59,1.62
75,0,5.74,1.64,0.0,5.59,1.62,0.0,5.44,1.56
105,0,5.44,1.56,0.0,5.30,1.54,0.0,5.26,1.82
120,0,5.30,1.54,0.0,5.26,1.82,0.0,5.23,1.86
135,0,5.26,1.82,0.0,5.23,1.86,0.0,5.21,1.89
150,0,5.23,1.86,0.0,5.21,1.89,1.0,5.20,1.20
